In [3]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd


In [65]:
DATA = pd.read_csv("area_of_work_prediction2.csv")
DATA.head()


,domain,keywords,company_name,not_mapped,mapping_area,prediction
0,https://1library.net,"['research and development', 'quality assuranc...",1library,"['gdp', 'api']","{'research and development': ['DS'], 'quality ...","{'DP': 0.3462, 'DS': 0.5769, 'AD': 0.0769}"
1,https://3pbio.com,"['recombinant proteins', 'clinical manufacturi...",3pbio,"['gmp', 'cgmp']","{'recombinant proteins': ['DS'], 'clinical man...","{'DP': 0.1818, 'DS': 0.8182, 'AD': 0.0}"
2,https://908devices.com,"['capillary electrophoresis', 'mammalian cell ...",908devices,['ms'],"{'capillary electrophoresis': ['AD'], 'mammali...","{'DP': 0.1154, 'DS': 0.6538, 'AD': 0.2308}"
3,https://99bitcoins.com,"['solution', 'research', 'development', 'teste...",99bitcoins,"['cex', 'ad', 'sec']","{'solution': ['DP'], 'research': ['DS'], 'deve...","{'DP': 0.4167, 'DS': 0.5833, 'AD': 0.0}"
4,https://abzena.com,"['cell line development', 'mammalian cell cult...",Abzena,[],"{'cell line development': ['DS'], 'mammalian c...","{'DP': 0.0667, 'DS': 0.9333, 'AD': 0.0}"


In [237]:
out = widgets.Output()  # (layout={'border': '1px solid black'})


In [238]:
# params = [(k, v) for v, k in enumerate(DATA['domain'])]


In [332]:
wlist = widgets.SelectMultiple(
    options=[], value=[], rows=10, description="Keywords", disabled=False
)

w_not_mapped_keywords = widgets.SelectMultiple(
    options=[], value=[], rows=10, description="Not mapped keywords", disabled=False
)


In [240]:
def test_keyword_match(row, keyword):
    _keywords = eval(row["keywords"])
    return any([keyword in _keyword for _keyword in _keywords])


In [380]:
# DATA.apply(test_keyword_match, axis=1, args=('resear', ))


In [381]:
def show_domain(Domain=params):
    predictions = eval(DATA.iloc[Domain]["prediction"])
    kwords = eval(DATA.iloc[Domain]["keywords"])
    wlist.options = kwords
    name = DATA.iloc[Domain]["company_name"]
    text = list(predictions.keys())
    values = list(predictions.values())
    with out:
        out.clear_output()
        plt.cla()  # Clearing the ax
        plt.bar(list(range(len(values))), values, edgecolor="k")
        for idx, t in enumerate(text):
            plt.text(
                idx,
                values[idx],
                f"{100*values[idx]:.0f}%",
                va="bottom",
                ha="center",
                fontsize=16,
            )
        plt.xticks(list(range(len(values))), [s.upper() for s in text], fontsize=16)
        plt.ylim(0, 1)
        plt.yticks([])
        plt.title(name)
        plt.show("off")


def show_domain_():
    Domain = w_domains_list.value
    sub = DATA["domain"] == Domain
    if sum(sub) > 0:
        predictions = eval(DATA.loc[sub, "prediction"].to_list()[0])
        kwords = eval(DATA.loc[sub, "keywords"].to_list()[0])
        wlist.options = kwords
        name = DATA.loc[sub, "company_name"].to_list()[0]
        text = list(predictions.keys())
        values = list(predictions.values())
        pred = text[values.index(max(values))]
        with out:
            out.clear_output()
            print("Company name: ", name)
            print("Predicted activity: ", pred.upper())
            plt.cla()  # Clearing the ax
            plt.bar(list(range(len(values))), values, edgecolor="k")
            for idx, t in enumerate(text):
                plt.text(
                    idx,
                    values[idx],
                    f"{100*values[idx]:.0f}%",
                    va="bottom",
                    ha="center",
                    fontsize=16,
                )
            plt.xticks(list(range(len(values))), [s.upper() for s in text], fontsize=16)
            plt.ylim(0, 1)
            plt.yticks([])
            plt.show("off")
    else:
        clear_all()


btn_approve = widgets.Button(
    description="Approve domain",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Approve!",
    layout=widgets.Layout(width="70%"),
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
btn_reject = widgets.Button(
    description="To blacklist",
    disabled=False,
    button_style="danger",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Reject!",
    layout=widgets.Layout(width="70%"),
    icon="ban",  # (FontAwesome names without the `fa-` prefix)
)


def approve(*args):
    pass


def reject(*args):
    pass


btn_approve.on_click(approve)
btn_reject.on_click(reject)


In [382]:
def filter_by_keyword(*args):
    clear_all()
    filter_keyword = w_filter_text.value
    if len(filter_keyword) >= 3:
        subset = DATA.apply(test_keyword_match, axis=1, args=(filter_keyword,))
        filtered_domains = DATA[subset]["domain"]
        w_domains_list.options = filtered_domains
    else:
        w_domains_list.options = DATA["domain"].to_list()


In [383]:
w_filter_text = widgets.Text(
    value="", placeholder="filter keywords...", description="Filter", disabled=False
)
btn_filter = button = widgets.Button(
    description="Filter!",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to filter",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
btn_filter.on_click(filter_by_keyword)
w_filter = widgets.HBox([w_filter_text, btn_filter])


In [392]:
w_domains_list = widgets.Dropdown(
    options=domains_list, value=domains_list[0], description="Domain"
)


def on_change(change):
    if change["type"] == "change":
        show_domain_()


w_domains_list.observe(on_change)


In [393]:
def clear_all():
    wlist.options = []
    with out:
        out.clear_output()


In [394]:
rLayout = widgets.Layout(align_items="flex-end")


In [395]:
w_predictions = widgets.VBox(
    [
        w_filter,
        widgets.HBox(
            [
                widgets.VBox(
                    [w_domains_list, wlist, btn_approve, btn_reject], layout=rLayout
                ),
                out,
            ]
        ),
    ]
)


In [396]:
# w_predictions = widgets.VBox([w_filter, w, widgets.HBox([out, widgets.VBox([wlist, widgets.HBox([btn_approve, btn_reject])])])])


In [397]:
w_predictions


In [72]:
# manual parse of the string into dictionary
text = DATA.iloc[0]["predictions"]
tmp = {}
for s in text.replace("}", "").split(","):
    key = s.split("'")[1]
    val = float(s.split(":")[1].strip())
    tmp[key] = val


# Domains blacklist

In [73]:
domains_blacklist = pd.read_csv("domain_blacklist.txt", names=["domain"])[
    "domain"
].to_list()


In [74]:
# def remove_from_blacklist()


In [75]:
blist = widgets.SelectMultiple(
    options=domains_blacklist, value=[], rows=10, description="Domains", disabled=False
)
domain_add_to_blacklist = widgets.Text(
    value="", placeholder="...add domain...", description="", disabled=False
)


In [76]:
btn_add_to_blacklist = widgets.Button(
    description="Add to blacklist",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Add",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
btn_remove_from_blacklist = widgets.Button(
    description="Remove from blacklist",
    disabled=False,
    button_style="info",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Remove",
    layout=widgets.Layout(width="95%", alignment="right"),
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)


def add_to_blacklist(*args):
    new = domain_add_to_blacklist.value
    if len(new) > 0 and "." in new:
        options = list(blist.options)
        if new not in options:
            options.append(new)
            options.sort()
            blist.options = options


def remove_from_blacklist(*args):
    selected = blist.value
    if len(selected) > 0:
        blist.options = [sel for sel in blist.options if sel not in selected]


btn_remove_from_blacklist.on_click(remove_from_blacklist)
btn_add_to_blacklist.on_click(add_to_blacklist)


In [77]:
w_blacklist = widgets.HBox(
    [
        widgets.VBox([blist, btn_remove_from_blacklist]),
        widgets.VBox([domain_add_to_blacklist, btn_add_to_blacklist]),
    ],
    box_style="warning",
)


In [78]:
w_blacklist


HBox(box_style='warning', children=(VBox(children=(SelectMultiple(description='Domains', options=('*.acs.org',…

In [79]:
tab_contents = ["P0", "P1", "P2", "P3", "P4"]
children = [w_predictions, w_blacklist]
tab = widgets.Tab()
tab.children = children
tab.titles = (
    "Company_activities",
    "Domain blacklist",
)  # does not work, wtf
tab
